# Modelado: Cola de Migraciones

Basándonos en la flota de Aerolíneas Argentinas, definimos los siguientes modelos de avión a modelar:

*   Embraer: capacidad de 96 pasajeros
*   Boeing 737: capacidad de 170 pasajeros
*   Airbus: capacidad de 220 pasajeros

En cada caso, estimamos que la ocupación con una lognormal centrada en la media 5% para representar la ocupación estimada promedio. Se truncan números mayores a la capacidad máxima.

Vuelos Internacionales: https://mensajero.com.ar/aereas/aeropuertos-argentina--se-espera-un-8---mas-de-pasajeros-en-aeroparque_a669027f81448034e37ec7055


## Modelado

In [ ]:
# -*- coding: utf-8 -*-
import time, json, datetime as dt
from typing import Tuple, List, Optional, Dict
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import matplotlib.animation as animation
from IPython.display import HTML
from collections import Counter
import random

In [ ]:
class Avion:
    """Representa un único avión en la simulación."""
    def __init__(self, id_avion, minuto_actual, pasajeros, tipo):
        self.id = id_avion
        self.distancia_a_aep = 100.0
        self.velocidad_actual = 0.0
        self.estado = "APROXIMANDO"
        self.minuto_creacion = minuto_actual
        self.pasajeros = pasajeros
        self.tipo = tipo

    def __repr__(self):
        """Formato para imprimir el estado del avión de forma legible."""
        return (f"Avion(ID:{self.id}, Dist:{self.distancia_a_aep:.1f}mn, "
                f"Vel:{self.velocidad_actual:.0f}kt, Est:{self.estado})")

In [ ]:
MODELOS = {
    "Embraer": 96,
    "Boeing 737": 170,
    "Airbus": 220
}

VENTANILLAS_MIGRACIONES = 15 # cantidad de ventanillas para migraciones
TERMINALES_MIGRACIONES = 8 # cantidad de terminales electrónicas
TASA_MIGRACIONES_MANUAL = 4  # minutos por persona
TASA_MIGRACIONES_AUTOMATICA = 1  # minutos por persona

In [ ]:
class Migraciones:
    def __init__(self, tipo, tiempo_servicio):
        self.tipo = tipo
        self.tiempo_servicio = tiempo_servicio
        self.ocupado_hasta = 0

    def libre(self, t):
        return t >= self.ocupado_hasta

    def asignar(self, t):
        """
        Asigna un pasajero a este puesto en minuto t.
        Devuelve el tiempo estimado que tardará ese pasajero en terminar.
        """
        tiempo_actual = self.tiempo_servicio
        if self.tipo == "kiosco" and random.random() < 0.25:
            tiempo_actual *= 2  # falla técnica: tarda doble

        self.ocupado_hasta = t + tiempo_actual
        return tiempo_actual  # tiempo que estará ocupado este pasajero


In [ ]:
class SistemaMigraciones:
    def __init__(self, prob_auto=0.5):
        """
        prob_auto: probabilidad que un pasajero doméstico prefiera kiosco (si está libre).
        """
        self.puestos = [Migraciones("manual", TASA_MIGRACIONES_MANUAL) for _ in range(VENTANILLAS_MIGRACIONES)] + \
                       [Migraciones("kiosco", TASA_MIGRACIONES_AUTOMATICA) for _ in range(TERMINALES_MIGRACIONES)]
        
        # colas separadas
        self.cola_domesticos = []       # tiempos de llegada de pasajeros domésticos
        self.cola_internacionales = []  # tiempos de llegada de pasajeros internacionales
        self.prob_auto = prob_auto

        # tiempos de espera
        self.tiempos_espera = []            # todos juntos
        self.tiempos_domesticos = []
        self.tiempos_internacionales = []

    def add_arrivals(self, n, minuto_actual, tipo="domestico"):
        """
        Agrega pasajeros a la cola correspondiente.
        tipo: "domestico" o "internacional".
        """
        if tipo == "domestico":
            self.cola_domesticos.extend([minuto_actual]*n)
        else:
            self.cola_internacionales.extend([minuto_actual]*n)

    def step(self, minuto):
        libres = [s for s in self.puestos if s.libre(minuto)]
        random.shuffle(libres)
        asignados = 0

        # --- Primero atender internacionales (solo ventanilla manual) ---
        while self.cola_internacionales and any(s.tipo == "manual" for s in libres):
            llegada = self.cola_internacionales.pop(0)
            manuales_libres = [s for s in libres if s.tipo == "manual"]
            puesto = random.choice(manuales_libres)
            tiempo_ocupado = puesto.asignar(minuto)

            espera = minuto + tiempo_ocupado - llegada
            self.tiempos_espera.append(espera)
            self.tiempos_internacionales.append(espera)

            libres.remove(puesto)
            asignados += 1

        # --- Luego atender domésticos ---
        while self.cola_domesticos and libres:
            llegada = self.cola_domesticos.pop(0)
            if random.random() < self.prob_auto:
                candidatos = [s for s in libres if s.tipo == "kiosco"]
            else:
                candidatos = [s for s in libres if s.tipo == "manual"]
            if not candidatos:
                candidatos = libres
            puesto = random.choice(candidatos)
            tiempo_ocupado = puesto.asignar(minuto)

            espera = minuto + tiempo_ocupado - llegada
            self.tiempos_espera.append(espera)
            self.tiempos_domesticos.append(espera)

            libres.remove(puesto)
            asignados += 1

        en_servicio = len(self.puestos) - sum(s.libre(minuto) for s in self.puestos)
        return {
            "minuto": minuto,
            "cola_domesticos": len(self.cola_domesticos),
            "cola_internacionales": len(self.cola_internacionales),
            "en_cola": len(self.cola_domesticos) + len(self.cola_internacionales),
            "en_servicio": en_servicio,
            "asignados": asignados
        }

    def promedio_espera(self, tipo=None):
        """
        Devuelve el tiempo promedio de espera:
          - tipo="domestico": solo domésticos
          - tipo="internacional": solo internacionales
          - tipo=None: todos
        """
        if tipo == "domestico":
            return sum(self.tiempos_domesticos)/len(self.tiempos_domesticos) if self.tiempos_domesticos else 0
        elif tipo == "internacional":
            return sum(self.tiempos_internacionales)/len(self.tiempos_internacionales) if self.tiempos_internacionales else 0
        else:
            return sum(self.tiempos_espera)/len(self.tiempos_espera) if self.tiempos_espera else 0


In [ ]:
def calcular_velocidad_maxima_permitida(distancia):
    """Devuelve la velocidad máxima según la distancia a AEP, en kt."""
    if distancia > 50: return 300
    elif distancia > 15: return 250
    elif distancia > 5: return 200
    else: return 150

def calcular_velocidad_minima_permitida(distancia):
    """Devuelve la velocidad mínima según la distancia a AEP, en kt."""
    if distancia > 50: return 250
    elif distancia > 15: return 200
    elif distancia > 5: return 150
    else: return 120

def calcular_separacion_en_tiempo(avion_atras, avion_adelante):
    """Calcula la separación en minutos entre dos aviones."""
    distancia_separacion = avion_atras.distancia_a_aep - avion_adelante.distancia_a_aep
    velocidad_referencia = avion_atras.velocidad_actual
    if velocidad_referencia == 0:
        return float('inf')
    return (distancia_separacion / velocidad_referencia) * 60

def distancia_aeroparque_free_flow(posicion, margen_minutos) -> float:
    """ Calcula cuánta distancia se recorre en margen_minutos desde posicion hacia AEP."""
    distancia_recorrida = calcular_velocidad_maxima_permitida(posicion) / 60 * margen_minutos
    return distancia_recorrida


def buscar_gap(avion, fila_aviones) -> bool:
    """ Indica si el avión puede reinsertarse con un gap de 10 minutos o no. """
    distancia_actual = avion.distancia_a_aep
    # Calculo cuánta distancia recorro desde mi posición actual yendo para adelante o atrás 5 min,
    # asumiento que se mantiene la velocidad máxima del tramo actual
    distancia_recorrida = distancia_aeroparque_free_flow(distancia_actual, margen_minutos=5)

    # Revisar que de posicion hacia +- distancia recorrida no haya aviones APROXIMANDO
    for otro_avion in fila_aviones:
      if otro_avion.estado == "APROXIMANDO" or otro_avion.estado == "AJUSTANDO_VELOCIDAD":
        # Tiene que estar entre: distancia_actual + distancia_recorrida >= otro_avion.distancia_a_aep >= distancia_actual - distancia_recorrida
        if distancia_actual + distancia_recorrida >= otro_avion.distancia_a_aep >= distancia_actual - distancia_recorrida:
          return False
    return True

In [ ]:
def gestionar_logica_aproximacion(avion, avion_de_adelante):
    """Toma todas las decisiones para un avión que está APROXIMANDO."""
    if avion_de_adelante is None:
        # Caso 1: Camino libre. Va a máxima velocidad.
        avion.estado = "APROXIMANDO"
        avion.velocidad_actual = calcular_velocidad_maxima_permitida(avion.distancia_a_aep)
    else:
        tiempo_sep = calcular_separacion_en_tiempo(avion, avion_de_adelante)

        if tiempo_sep < 4:
            # Caso 2: Muy cerca. Intenta reducir velocidad.

            velocidad_req = avion_de_adelante.velocidad_actual - 20

            if velocidad_req < calcular_velocidad_minima_permitida(avion.distancia_a_aep):
                # Caso 3: No puede reducir lo suficiente. Se da la vuelta.
                avion.estado = "REGRESANDO"
                avion.velocidad_actual = 200
                print(f" (!) Avion {avion.id} inicia maniobra de regreso.")
            else:
                # Puede reducir de forma segura.
                avion.estado = "AJUSTANDO_VELOCIDAD"
                avion.velocidad_actual = velocidad_req
        else:
            # Hay espacio suficiente. Va a máxima velocidad.
            avion.estado = "APROXIMANDO"
            avion.velocidad_actual = calcular_velocidad_maxima_permitida(avion.distancia_a_aep)

def gestionar_logica_regreso(avion_regresando, todos_los_aviones, resultados=None, minuto=None, warmup=0):
    """
    Gestiona un avión REGRESANDO. Primero busca un hueco para reingresar.
    Si no lo encuentra, chequea si debe desviarse.
    """
    # 1. Si el bucle termina sin encontrar hueco, se chequea el desvío
    if avion_regresando.distancia_a_aep > 100:
        avion_regresando.estado = "DESVIADO"
        print(f" (X) Avion {avion_regresando.id} no encontró hueco y se desvió a Montevideo.")

    # 2. Buscar un hueco de 10 minutos entre cada par de aviones
    elif buscar_gap(avion_regresando, todos_los_aviones):
        # 3. Si se encuentra un hueco, el avión reingresa
        avion_regresando.estado = "APROXIMANDO"
        print(f" (O) ¡Avion {avion_regresando.id} encontró un hueco y reingresa a la fila!")

In [ ]:
def generar_nuevos_aviones(minuto, aviones, next_id, probabilidad):
    """Decide si un nuevo avión aparece en el horizonte."""
    if np.random.uniform(0,1) < probabilidad:
        # Generamos el modelo de avión con su cantidad de pasajeros
        modelo = np.random.choice(list(MODELOS.keys()))
        capacidad = MODELOS[modelo]

        # Generamos cantidad de pasajeros como lognormal centrada en 0.95 * capacidad
        pasajeros_a_bordo = min(int(np.random.lognormal(mean=np.log(0.95 * capacidad), sigma=5)), capacidad)

        nuevo_avion = Avion(next_id, minuto, pasajeros_a_bordo)
        aviones.append(nuevo_avion)
        print(f"Min {minuto}: -> Aparece Avion {next_id} con {pasajeros_a_bordo} pasajeros en el horizonte.")
        return next_id + 1
    return next_id

def actualizar_estados_y_velocidades(aviones, resultados=None, minuto=None, warmup=0):
    """Actualiza el estado y la velocidad de cada avión según las reglas."""
    aviones.sort(key=lambda avion: avion.distancia_a_aep)
    for i, avion in enumerate(aviones):
        if avion.estado == "REGRESANDO":
            gestionar_logica_regreso(avion, aviones, resultados, minuto, warmup)

        else: # APROXIMANDO o AJUSTANDO_VELOCIDAD
            avion_de_adelante = aviones[i-1] if i > 0 and aviones[i-1].estado != "REGRESANDO" else None
            gestionar_logica_aproximacion(avion, avion_de_adelante)

        if resultados is not None and avion.estado == "AJUSTANDO_VELOCIDAD" and minuto is not None and minuto >= warmup:
            resultados['congestion_events'] += 1

def mover_aviones(aviones):
    """Mueve cada avión según su velocidad final para este minuto."""
    for avion in aviones:
        distancia_recorrida = avion.velocidad_actual / 60
        if avion.estado == "REGRESANDO":
            avion.distancia_a_aep += distancia_recorrida
        else:
            avion.distancia_a_aep -= distancia_recorrida

def gestionar_aviones_finalizados(minuto, aviones, resultados=None, tiempo_ideal=23.4, warmup=0):
    aviones_activos = []
    aviones_finalizados_hoy = []
    for avion in aviones:
        if avion.distancia_a_aep <= 0:
            avion.estado = "ATERRIZADO"
            # Solo calculamos y contamos si nos pasaron el diccionario de resultados
            if resultados is not None and minuto is not None and minuto >= warmup:
                tiempo_real = minuto - avion.minuto_creacion
                retraso = max(0, tiempo_real - tiempo_ideal)
                resultados['total_delay_min'] += retraso
            aviones_finalizados_hoy.append(avion)
        elif avion.estado == "DESVIADO":
            aviones_finalizados_hoy.append(avion)
        else:
            aviones_activos.append(avion)
    return aviones_activos, aviones_finalizados_hoy

def guardar_datos_del_minuto(minuto, aviones, historial):
    """Guarda el estado de cada avión en el minuto actual."""
    for avion in aviones:
        historial.append({
            'minuto': minuto,
            'id': avion.id,
            'distancia': avion.distancia_a_aep,
            'estado': avion.estado
        })

## Simulación principal

In [ ]:
def simular_una_corrida(lambda_val, tiempo_total=1080, warmup=0):
    """
    Ejecuta una corrida completa de la simulación para un lambda dado
    y devuelve un diccionario con los resultados.
    """
    # --- Parámetros y Almacenamiento ---
    aviones_activos = []
    aviones_finalizados = []
    sistema_migraciones = SistemaMigraciones(prob_auto=0.66)
    next_avion_id = 1
    TIEMPO_IDEAL_VIAJE = 23.4  # Nuestro baseline en minutos

    # --- Contadores para ESTA corrida específica ---
    resultados_de_la_corrida = {
        'total_delay_min': 0.0,
        'congestion_events': 0,
        'diversions': 0,
        'landed_planes': 0,
        'inbound_planes': 0,
        'total_planes': 0,
        'active_planes':0,
        'returning_planes':0,
        'queue_size':0, 
        'queue_size_dom':0,
        'queue_size_int':0,
        'wait_time_avg':0.0,
        'wait_time_dom':0.0,
        'wait_time_int':0.0
    }

    historial_metricas = {
        'minuto': [],
        'delay_acum': [],
        'congestion_events': [],
        'diversions': [],
        'landed_planes': [],
        'delay_avg': [],
        'inbound_planes': [],
        'active_planes':[],
        'returning_planes':[],
        'queue_size':[],
        'queue_size_dom':[],
        'queue_size_int':[]
    }

    # --- Bucle de un día de simulación ---
    for minuto in range(tiempo_total):
        # Generar arribos (ahora pasa el minuto de creación)
        if random.random() < lambda_val:
            # Generamos el modelo de avión con su cantidad de pasajeros
            modelo = np.random.choice(list(MODELOS.keys()))
            capacidad = MODELOS[modelo]

            # Generamos cantidad de pasajeros como lognormal centrada en 0.95 * capacidad
            pasajeros_a_bordo = min(int(np.random.lognormal(mean=np.log(0.95 * capacidad), sigma=5)), capacidad)

            # Nacional o internacional
            tipo = np.random.choice(["domestico", "internacional"], p=[0.74, 0.26])
            nuevo_avion = Avion(next_avion_id, minuto, pasajeros_a_bordo, tipo)
            aviones_activos.append(nuevo_avion)
            next_avion_id += 1
            if minuto >= warmup:
                resultados_de_la_corrida['total_planes'] += 1


        # Actualizar estados y contar congestiones
        actualizar_estados_y_velocidades(aviones_activos, resultados_de_la_corrida, minuto=minuto, warmup=warmup)

        # Mover aviones
        mover_aviones(aviones_activos)

        # Gestionar aviones que terminaron (y contar retrasos, desvíos, aterrizajes)
        activos_actualizados, finalizados_ahora = gestionar_aviones_finalizados(
            minuto, aviones_activos, resultados_de_la_corrida, TIEMPO_IDEAL_VIAJE, warmup=warmup
        )
        aviones_activos = activos_actualizados
        aviones_finalizados.extend(finalizados_ahora)

        # Bajar pasajeros a la cola de migraciones
        for avion in finalizados_ahora:
            if avion.estado == "ATERRIZADO":
                sistema_migraciones.add_arrivals(avion.pasajeros, minuto, avion.tipo)

        # Actualizar sistema de migraciones una vez por minuto (asignaciones a puestos)
        migr_stats = sistema_migraciones.step(minuto)

        # Mantener métricas en resultados/historial (igual que antes)
        cola_migraciones = migr_stats['en_cola']
        if minuto >= warmup:
            historial_metricas['queue_size'].append(cola_migraciones)
            historial_metricas['queue_size_dom'].append(migr_stats['cola_domesticos'])
            historial_metricas['queue_size_int'].append(migr_stats['cola_internacionales'])
        else:
            historial_metricas['queue_size'].append(0)
            historial_metricas['queue_size_dom'].append(0)
            historial_metricas['queue_size_int'].append(0)

        # Guardar métricas del minuto (solo si pasó el warm-up)
        if minuto >= warmup:
            for avion in aviones_activos:
                if avion.estado == "APROXIMANDO" or avion.estado == "AJUSTANDO_VELOCIDAD":
                    resultados_de_la_corrida['inbound_planes'] += 1
                elif avion.estado == "REGRESANDO":
                    resultados_de_la_corrida['returning_planes'] += 1

            for avion in finalizados_ahora:
                if avion.estado == "DESVIADO":
                    resultados_de_la_corrida['diversions'] += 1
                elif avion.estado == "ATERRIZADO":
                    resultados_de_la_corrida['landed_planes'] += 1

        historial_metricas['minuto'].append(minuto)
        historial_metricas['delay_acum'].append(resultados_de_la_corrida['total_delay_min'])
        historial_metricas['congestion_events'].append(resultados_de_la_corrida['congestion_events'])
        historial_metricas['diversions'].append(resultados_de_la_corrida['diversions'])
        historial_metricas['landed_planes'].append(resultados_de_la_corrida['landed_planes'])
        landed = resultados_de_la_corrida['landed_planes']
        if landed > 0:
            avg_delay = resultados_de_la_corrida['total_delay_min'] / landed
        else:
            avg_delay = 0.0
        historial_metricas['delay_avg'].append(avg_delay)
        historial_metricas['inbound_planes'].append(resultados_de_la_corrida['inbound_planes'])
        historial_metricas['active_planes'].append(len(aviones_activos))
        historial_metricas['returning_planes'].append(resultados_de_la_corrida['returning_planes'])

    resultados_de_la_corrida['active_planes'] = sum(historial_metricas['active_planes'])
    resultados_de_la_corrida['queue_size'] = np.mean(historial_metricas['queue_size'])
    resultados_de_la_corrida['queue_size_dom'] = np.mean(historial_metricas['queue_size_dom'])
    resultados_de_la_corrida['queue_size_int'] = np.mean(historial_metricas['queue_size_int'])
    resultados_de_la_corrida['wait_time_avg'] = sistema_migraciones.promedio_espera()
    resultados_de_la_corrida['wait_time_dom'] = sistema_migraciones.promedio_espera("domestico")
    resultados_de_la_corrida['wait_time_int'] = sistema_migraciones.promedio_espera("internacional")


    return resultados_de_la_corrida, pd.DataFrame(historial_metricas)

def ejecutar_experimentos():
    """
    Función principal que ejecuta el bucle experimental para varios lambdas.
    """
    # --- Configuración del Experimento ---
    lambdas_a_probar = [0.02, 0.1, 0.2, 0.5, 1.0]
    N_REPETICIONES = 100 # Número de veces que se repite la simulación para cada lambda
    WARMUP_MIN = 60

    resultados_finales = []
    historiales = []
    print("--- Iniciando Bucle Experimental ---")
    for lambda_val in lambdas_a_probar:
        print(f"\n--- Probando con λ = {lambda_val:.4f} ---")
        for i in range(N_REPETICIONES):
            resultado_run, df_hist = simular_una_corrida(lambda_val, warmup=WARMUP_MIN)
            resultado_run['lambda'] = lambda_val
            resultados_finales.append(resultado_run)
            df_hist['lambda'] = lambda_val
            df_hist['rep'] = i
            historiales.append(df_hist)
            print(".", end="")
    print("\n\n--- Bucle Experimental Finalizado ---")
    df_resultados = pd.DataFrame(resultados_finales)
    df_historiales = pd.concat(historiales, ignore_index=True)
    return df_resultados, df_historiales

if __name__ == '__main__':
    np.random.seed(42)
    random.seed(42)
    df_final_resultados, df_hist = ejecutar_experimentos()

In [ ]:
df_estadisticas = df_final_resultados.groupby('lambda').agg(
    n_simulaciones=('lambda', 'count'),
    total_aviones_promedio=('total_planes', 'mean'),
    cola_migraciones_promedio=('queue_size', 'mean'),
    cola_migraciones_std=('queue_size', 'std'),
    tiempo_espera_promedio=('wait_time_avg', 'mean'),
    tiempo_espera_std=('wait_time_avg', 'std'),
    tiempo_espera_domesticos_promedio=('wait_time_dom', 'mean'),
    tiempo_espera_internacionales_promedio=('wait_time_int', 'mean'),
    tiempo_espera_internacionales_std=('wait_time_int', 'std'),
    tiempo_espera_domesticos_std=('wait_time_dom', 'std'),
    cola_migraciones_domesticos_promedio=('queue_size_dom', 'mean'),
    cola_migraciones_domesticos_std=('queue_size_dom', 'std'),
    cola_migraciones_internacionales_promedio=('queue_size_int', 'mean'),
    cola_migraciones_internacionales_std=('queue_size_int', 'std')
)
# 2. Calcular el Error Estándar de la Media (nuestro error de estimación)
df_estadisticas['tiempo_espera_error'] = 1.96 * df_estadisticas['tiempo_espera_std'] / np.sqrt(df_estadisticas['n_simulaciones'])
df_estadisticas['tiempo_espera_domesticos_error'] = 1.96 * df_estadisticas['tiempo_espera_domesticos_std'] / np.sqrt(df_estadisticas['n_simulaciones'])
df_estadisticas['tiempo_espera_internacionales_error'] = 1.96 * df_estadisticas['tiempo_espera_internacionales_std'] / np.sqrt(df_estadisticas['n_simulaciones'])
df_estadisticas['cola_migraciones_domesticos_error'] = 1.96 * df_estadisticas['cola_migraciones_domesticos_std'] / np.sqrt(df_estadisticas['n_simulaciones'])
df_estadisticas['cola_migraciones_internacionales_error'] = 1.96 * df_estadisticas['cola_migraciones_internacionales_std'] / np.sqrt(df_estadisticas['n_simulaciones'])
df_estadisticas['cola_migraciones_error'] = 1.96 * df_estadisticas['cola_migraciones_std'] / np.sqrt(df_estadisticas['n_simulaciones'])

# Eliminar atraso y stds
df_estadisticas = df_estadisticas.drop(columns=['cola_migraciones_std', 'tiempo_espera_std',
                                                 'tiempo_espera_domesticos_std', 'tiempo_espera_internacionales_std',
                                                 'cola_migraciones_domesticos_std', 'cola_migraciones_internacionales_std'])

# Mostramos la tabla final de resultados
print("--- Tabla de Estadísticas por Valor de λ ---")
display(df_estadisticas)

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(
    df_estadisticas.index.astype(str),
    df_estadisticas['cola_migraciones_promedio'],
    yerr=df_estadisticas['cola_migraciones_error'],
    capsize=5,
    color='wheat'
)
plt.title('Tamaño de la cola de migraciones vs. Tasa de Arribo (λ)', fontsize=14)
plt.xlabel('Valor de λ (Probabilidad de arribo por minuto)', fontsize=12)
plt.ylabel('Cola promedio de migraciones por minuto', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Graficar cola de migraciones domésticos e internacionales
plt.figure(figsize=(10, 6))
width = 0.35  # Ancho de las barras
x = np.arange(len(df_estadisticas.index))   # Posiciones en el eje x   
plt.bar(
    x - width/2,
    df_estadisticas['cola_migraciones_domesticos_promedio'],
    width,
    yerr=df_estadisticas['cola_migraciones_domesticos_error'],
    capsize=5,
    label='Domésticos',
    color='lightblue'
)
plt.bar(
    x + width/2,
    df_estadisticas['cola_migraciones_internacionales_promedio'],
    width,
    yerr=df_estadisticas['cola_migraciones_internacionales_error'],
    capsize=5,
    label='Internacionales',
    color='salmon'
)
plt.title('Tamaño de la cola de migraciones por tipo de pasajero vs. Tasa de Arribo (λ)', fontsize=14)
plt.xlabel('Valor de λ (Probabilidad de arribo por minuto)', fontsize=12)
plt.ylabel('Cola promedio de migraciones por minuto', fontsize=12)
plt.xticks(ticks=x, labels=df_estadisticas.index.astype(str))
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Gráfico tiempo espera migraciones vs lambda
plt.figure(figsize=(10, 6))
plt.bar(
    df_estadisticas.index.astype(str),
    df_estadisticas['tiempo_espera_promedio'],
    yerr=df_estadisticas['tiempo_espera_error'],
    capsize=5,
    color='royalblue'
)
plt.title('Tiempo de espera en migraciones vs. Tasa de Arribo (λ)', fontsize=14)
plt.xlabel('Valor de λ (Probabilidad de arribo por minuto)', fontsize=12)
plt.ylabel('Tiempo de espera promedio en migraciones (minutos)', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Graficar tiempos de espera domésticos vs internacionales x lambda
plt.figure(figsize=(10, 6))
x = np.arange(len(df_estadisticas.index))
width = 0.35
plt.bar(x - width/2, df_estadisticas['tiempo_espera_domesticos_promedio'], width, label='Domésticos', color='skyblue',
        yerr=df_estadisticas['tiempo_espera_domesticos_error'], capsize=5   )
plt.bar(x + width/2, df_estadisticas['tiempo_espera_internacionales_promedio'], width, label='Internacionales', color='salmon',
        yerr=df_estadisticas['tiempo_espera_internacionales_error'], capsize=5)
plt.xticks(x, df_estadisticas.index.astype(str))
plt.title('Tiempo de Espera en Migraciones por Tipo de Pasajero vs. Tasa de Arribo (λ)', fontsize=14)
plt.xlabel('Valor de λ (Probabilidad de arribo por minuto)', fontsize=12)
plt.ylabel('Tiempo de Espera Promedio en Migraciones (minutos)', fontsize=12)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show() 

# Políticas

Si usamos alguna de las políticas propuestas, ¿cómo cambia esta métrica? ¿Qué impacto tiene en la cola en tierra?

## Política 1: reducción proporcional

In [ ]:
def gestionar_logica_aproximacion_politica_1(avion, avion_de_adelante, FACTOR_REDUCCION_VELOCIDAD=0.4):
    """Toma todas las decisiones para un avión que está APROXIMANDO."""
    if avion_de_adelante is None:
        # Caso 1: Camino libre. Va a máxima velocidad.
        avion.estado = "APROXIMANDO"
        avion.velocidad_actual = calcular_velocidad_maxima_permitida(avion.distancia_a_aep)
    else:
        tiempo_sep = calcular_separacion_en_tiempo(avion, avion_de_adelante)
        v_max = calcular_velocidad_maxima_permitida(avion.distancia_a_aep)
        v_min = calcular_velocidad_minima_permitida(avion.distancia_a_aep)
        rango_size = v_max - v_min 

        if tiempo_sep < 4:

            reduccion = FACTOR_REDUCCION_VELOCIDAD * rango_size 
            velocidad_req = avion.velocidad_actual - reduccion

            if velocidad_req < v_min:
                # Caso 3: No puede reducir lo suficiente. Se da la vuelta.
                avion.estado = "REGRESANDO"
                avion.velocidad_actual = 200
                print(f" (!) Avion {avion.id} inicia maniobra de regreso.")
            else:
                # Puede reducir de forma segura.
                avion.estado = "AJUSTANDO_VELOCIDAD"
                avion.velocidad_actual = velocidad_req
        else:
            # Hay espacio suficiente. Va a máxima velocidad.
            avion.estado = "APROXIMANDO"
            avion.velocidad_actual = calcular_velocidad_maxima_permitida(avion.distancia_a_aep)



def actualizar_estados_y_velocidades_politica_1(aviones, resultados=None, minuto=None, warmup=0):
    """Actualiza el estado y la velocidad de cada avión según las reglas."""
    aviones.sort(key=lambda avion: avion.distancia_a_aep)
    for i, avion in enumerate(aviones):
        if avion.estado == "REGRESANDO":
            gestionar_logica_regreso(avion, aviones, resultados, minuto, warmup)

        else: # APROXIMANDO o AJUSTANDO_VELOCIDAD
            avion_de_adelante = aviones[i-1] if i > 0 and aviones[i-1].estado != "REGRESANDO" else None
            gestionar_logica_aproximacion_politica_1(avion, avion_de_adelante, FACTOR_REDUCCION_VELOCIDAD=0.4)

        if resultados is not None and avion.estado == "AJUSTANDO_VELOCIDAD" and minuto is not None and minuto >= warmup:
            resultados['congestion_events'] += 1

In [ ]:
def simular_una_corrida_politica_1(lambda_val, tiempo_total=1080, warmup=0):
    """
    Ejecuta una corrida completa de la simulación para un lambda dado
    y devuelve un diccionario con los resultados.
    """
    # --- Parámetros y Almacenamiento ---
    aviones_activos = []
    aviones_finalizados = []
    next_avion_id = 1
    TIEMPO_IDEAL_VIAJE = 23.4  # Nuestro baseline en minutos
    sistema_migraciones = SistemaMigraciones(prob_auto=0.66) 
    

    # --- Contadores para ESTA corrida específica ---
    resultados_de_la_corrida = {
        'total_delay_min': 0.0,
        'congestion_events': 0,
        'diversions': 0,
        'landed_planes': 0,
        'inbound_planes': 0,
        'total_planes': 0,
        'active_planes':0,
        'returning_planes':0,
        'queue_size':0, 
        'queue_size_dom':0,
        'queue_size_int':0,
        'wait_time_avg':0.0,
        'wait_time_dom':0.0,
        'wait_time_int':0.0
    }

    historial_metricas = {
        'minuto': [],
        'delay_acum': [],
        'congestion_events': [],
        'diversions': [],
        'landed_planes': [],
        'delay_avg': [],
        'inbound_planes': [],
        'active_planes':[],
        'returning_planes':[],
        'queue_size':[],
        'queue_size_dom':[],
        'queue_size_int':[]
    }

    # --- Bucle de un día de simulación ---
    for minuto in range(tiempo_total):
        # Generar arribos (ahora pasa el minuto de creación)
        if random.random() < lambda_val:
            # Generamos el modelo de avión con su cantidad de pasajeros
            modelo = np.random.choice(list(MODELOS.keys()))
            capacidad = MODELOS[modelo]

            # Generamos cantidad de pasajeros como lognormal centrada en 0.95 * capacidad
            pasajeros_a_bordo = min(int(np.random.lognormal(mean=np.log(0.95 * capacidad), sigma=5)), capacidad)

            # Nacional o internacional
            tipo = np.random.choice(["domestico", "internacional"], p=[0.74, 0.26])
            nuevo_avion = Avion(next_avion_id, minuto, pasajeros_a_bordo, tipo)
            aviones_activos.append(nuevo_avion)
            next_avion_id += 1
            if minuto >= warmup:
                resultados_de_la_corrida['total_planes'] += 1

        # Actualizar estados y contar congestiones
        actualizar_estados_y_velocidades_politica_1(aviones_activos, resultados_de_la_corrida, minuto=minuto, warmup=warmup)

        # Mover aviones
        mover_aviones(aviones_activos)

        # Gestionar aviones que terminaron (y contar retrasos, desvíos, aterrizajes)
        activos_actualizados, finalizados_ahora = gestionar_aviones_finalizados(
            minuto, aviones_activos, resultados_de_la_corrida, TIEMPO_IDEAL_VIAJE, warmup=warmup
        )
        aviones_activos = activos_actualizados
        aviones_finalizados.extend(finalizados_ahora)

        # Bajar pasajeros a la cola de migraciones
        for avion in finalizados_ahora:
            if avion.estado == "ATERRIZADO":
                sistema_migraciones.add_arrivals(avion.pasajeros, minuto, avion.tipo)

        # Actualizar sistema de migraciones una vez por minuto (asignaciones a puestos)
        migr_stats = sistema_migraciones.step(minuto)

        # Mantener métricas en resultados/historial (igual que antes)
        cola_migraciones = migr_stats['en_cola']
        if minuto >= warmup:
            historial_metricas['queue_size'].append(cola_migraciones)
            historial_metricas['queue_size_dom'].append(migr_stats['cola_domesticos'])
            historial_metricas['queue_size_int'].append(migr_stats['cola_internacionales'])
        else:
            historial_metricas['queue_size'].append(0)
            historial_metricas['queue_size_dom'].append(0)
            historial_metricas['queue_size_int'].append(0)

        # Guardar métricas del minuto (solo si pasó el warm-up)
        if minuto >= warmup:
            for avion in aviones_activos:
                if avion.estado == "APROXIMANDO" or avion.estado == "AJUSTANDO_VELOCIDAD":
                    resultados_de_la_corrida['inbound_planes'] += 1
                elif avion.estado == "REGRESANDO":
                    resultados_de_la_corrida['returning_planes'] += 1
            
            for avion in finalizados_ahora:
                if avion.estado == "DESVIADO":
                    resultados_de_la_corrida['diversions'] += 1
                elif avion.estado == "ATERRIZADO":
                    resultados_de_la_corrida['landed_planes'] += 1

        historial_metricas['minuto'].append(minuto)
        historial_metricas['delay_acum'].append(resultados_de_la_corrida['total_delay_min'])
        historial_metricas['congestion_events'].append(resultados_de_la_corrida['congestion_events'])
        historial_metricas['diversions'].append(resultados_de_la_corrida['diversions'])
        historial_metricas['landed_planes'].append(resultados_de_la_corrida['landed_planes'])
        landed = resultados_de_la_corrida['landed_planes']
        if landed > 0:
            avg_delay = resultados_de_la_corrida['total_delay_min'] / landed
        else:
            avg_delay = 0.0
        historial_metricas['delay_avg'].append(avg_delay)
        historial_metricas['inbound_planes'].append(resultados_de_la_corrida['inbound_planes'])
        historial_metricas['active_planes'].append(len(aviones_activos))
        historial_metricas['returning_planes'].append(resultados_de_la_corrida['returning_planes'])
    
    resultados_de_la_corrida['active_planes'] = sum(historial_metricas['active_planes'])
    resultados_de_la_corrida['queue_size'] = np.mean(historial_metricas['queue_size'])
    resultados_de_la_corrida['queue_size_dom'] = np.mean(historial_metricas['queue_size_dom'])
    resultados_de_la_corrida['queue_size_int'] = np.mean(historial_metricas['queue_size_int'])
    resultados_de_la_corrida['wait_time_avg'] = sistema_migraciones.promedio_espera()
    resultados_de_la_corrida['wait_time_dom'] = sistema_migraciones.promedio_espera("domestico")
    resultados_de_la_corrida['wait_time_int'] = sistema_migraciones.promedio_espera("internacional")

    return resultados_de_la_corrida, pd.DataFrame(historial_metricas)

def ejecutar_experimentos_politica_1():
    """
    Función principal que ejecuta el bucle experimental para varios lambdas.
    """
    # --- Configuración del Experimento ---
    lambdas_a_probar = [0.02, 0.1, 0.2, 0.5, 1.0]
    N_REPETICIONES = 50 # Número de veces que se repite la simulación para cada lambda
    WARMUP_MIN = 60 

    resultados_finales = []
    historiales = []
    print("--- Iniciando Bucle Experimental ---")
    for lambda_val in lambdas_a_probar:
        print(f"\n--- Probando con λ = {lambda_val:.4f} ---")
        for i in range(N_REPETICIONES):
            resultado_run, df_hist = simular_una_corrida_politica_1(lambda_val, warmup=WARMUP_MIN)
            resultado_run['lambda'] = lambda_val
            resultados_finales.append(resultado_run)
            df_hist['lambda'] = lambda_val
            df_hist['rep'] = i
            historiales.append(df_hist)
            print(".", end="")
    print("\n\n--- Bucle Experimental Finalizado ---")
    df_resultados = pd.DataFrame(resultados_finales)
    df_historiales = pd.concat(historiales, ignore_index=True)
    return df_resultados, df_historiales

if __name__ == '__main__':
    np.random.seed(42)
    random.seed(42)
    df_final_resultados_politica_1, df_hist = ejecutar_experimentos_politica_1()

In [ ]:
df_estadisticas_politica_1 = df_final_resultados_politica_1.groupby('lambda').agg(
    n_simulaciones=('lambda', 'count'),
    total_aviones_promedio=('total_planes', 'mean'),
    cola_migraciones_promedio=('queue_size', 'mean'),
    cola_migraciones_std=('queue_size', 'std'),
    tiempo_espera_promedio=('wait_time_avg', 'mean'),
    tiempo_espera_std=('wait_time_avg', 'std'),
    tiempo_espera_domesticos_promedio=('wait_time_dom', 'mean'),
    tiempo_espera_internacionales_promedio=('wait_time_int', 'mean'),
    tiempo_espera_internacionales_std=('wait_time_int', 'std'),
    tiempo_espera_domesticos_std=('wait_time_dom', 'std'),
    cola_migraciones_domesticos_promedio=('queue_size_dom', 'mean'),
    cola_migraciones_domesticos_std=('queue_size_dom', 'std'),
    cola_migraciones_internacionales_promedio=('queue_size_int', 'mean'),
    cola_migraciones_internacionales_std=('queue_size_int', 'std')
)
# 2. Calcular el Error Estándar de la Media (nuestro error de estimación)
df_estadisticas_politica_1['tiempo_espera_error'] = 1.96 * df_estadisticas_politica_1['tiempo_espera_std'] / np.sqrt(df_estadisticas_politica_1['n_simulaciones'])
df_estadisticas_politica_1['tiempo_espera_domesticos_error'] = 1.96 * df_estadisticas_politica_1['tiempo_espera_domesticos_std'] / np.sqrt(df_estadisticas_politica_1['n_simulaciones'])
df_estadisticas_politica_1['tiempo_espera_internacionales_error'] = 1.96 * df_estadisticas_politica_1['tiempo_espera_internacionales_std'] / np.sqrt(df_estadisticas_politica_1['n_simulaciones'])
df_estadisticas_politica_1['cola_migraciones_domesticos_error'] = 1.96 * df_estadisticas_politica_1['cola_migraciones_domesticos_std'] / np.sqrt(df_estadisticas_politica_1['n_simulaciones'])
df_estadisticas_politica_1['cola_migraciones_internacionales_error'] = 1.96 * df_estadisticas_politica_1['cola_migraciones_internacionales_std'] / np.sqrt(df_estadisticas_politica_1['n_simulaciones'])
df_estadisticas_politica_1['cola_migraciones_error'] = 1.96 * df_estadisticas_politica_1['cola_migraciones_std'] / np.sqrt(df_estadisticas_politica_1['n_simulaciones'])

# Eliminar atraso y stds
df_estadisticas_politica_1 = df_estadisticas_politica_1.drop(columns=['cola_migraciones_std', 'tiempo_espera_std',
                                                 'tiempo_espera_domesticos_std', 'tiempo_espera_internacionales_std',
                                                 'cola_migraciones_domesticos_std', 'cola_migraciones_internacionales_std'])

# Mostramos la tabla final de resultados
print("--- Tabla de Estadísticas por Valor de λ ---")
display(df_estadisticas_politica_1)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True)
fig.suptitle('Impacto de implementación de política 1 en Migraciones', fontsize=18, weight='bold')

# --- Cola Domésticos ---
ax1 = axes[0,0]
ax1.plot(df_estadisticas.index, df_estadisticas['cola_migraciones_domesticos_promedio'], 
         marker='o', linestyle='--', color='royalblue', label='Día Normal')
ax1.plot(df_estadisticas_politica_1.index, df_estadisticas_politica_1['cola_migraciones_domesticos_promedio'], 
         marker='o', linestyle='-', color='deepskyblue', label='Con Política')
ax1.set_title('Cola Migraciones - Domésticos', fontsize=14)
ax1.set_ylabel('Tamaño Promedio de la Cola')
ax1.set_xlabel('Tasa de Arribo (λ)', fontsize=12)
ax1.legend()

# --- Cola Internacionales ---
ax2 = axes[0,1]
ax2.plot(df_estadisticas.index, df_estadisticas['cola_migraciones_internacionales_promedio'], 
         marker='s', linestyle='--', color='firebrick', label='Día Normal')
ax2.plot(df_estadisticas_politica_1.index, df_estadisticas_politica_1['cola_migraciones_internacionales_promedio'], 
         marker='s', linestyle='-', color='salmon', label='Con Política')
ax2.set_title('Cola Migraciones - Internacionales', fontsize=14)
ax2.set_ylabel('Tamaño Promedio de la Cola')
ax2.set_xlabel('Tasa de Arribo (λ)', fontsize=12)
ax2.legend()

# --- Espera Domésticos ---
ax3 = axes[1,0]
ax3.plot(df_estadisticas.index, df_estadisticas['tiempo_espera_domesticos_promedio'], 
         marker='o', linestyle='--', color='royalblue', label='Día Normal')
ax3.plot(df_estadisticas_politica_1.index, df_estadisticas_politica_1['tiempo_espera_domesticos_promedio'], 
         marker='o', linestyle='-', color='deepskyblue', label='Con Política')
ax3.set_title('Tiempo de Espera - Domésticos', fontsize=14)
ax3.set_ylabel('Minutos de Espera')
ax3.set_xlabel('Tasa de Arribo (λ)', fontsize=12)
ax3.legend()

# --- Espera Internacionales ---
ax4 = axes[1,1]
ax4.plot(df_estadisticas.index, df_estadisticas['tiempo_espera_internacionales_promedio'], 
         marker='s', linestyle='--', color='firebrick', label='Día Normal')
ax4.plot(df_estadisticas_politica_1.index, df_estadisticas_politica_1['tiempo_espera_internacionales_promedio'], 
         marker='s', linestyle='-', color='salmon', label='Con Política')
ax4.set_title('Tiempo de Espera - Internacionales', fontsize=14)
ax4.set_ylabel('Minutos de Espera')
ax4.set_xlabel('Tasa de Arribo (λ)', fontsize=12)
ax4.legend()

plt.tight_layout(rect=[0, 0, 1, 0.96])  # deja espacio para el título general
plt.show()


## Política 2: velocidad mínima

In [ ]:
def gestionar_logica_aproximacion_politica_2(avion, avion_de_adelante):
    """Toma todas las decisiones para un avión que está APROXIMANDO."""
    if avion_de_adelante is None:
        # Caso 1: Camino libre. Va a máxima velocidad.
        avion.estado = "APROXIMANDO"
        avion.velocidad_actual = calcular_velocidad_maxima_permitida(avion.distancia_a_aep)
    else:
        tiempo_sep = calcular_separacion_en_tiempo(avion, avion_de_adelante)

        velocidad_minima = calcular_velocidad_minima_permitida(avion.distancia_a_aep)

        if tiempo_sep < 4:
            if avion.velocidad_actual <= velocidad_minima:
                # Caso 2: No puede reducir lo suficiente. Se da la vuelta.
                avion.estado = "REGRESANDO"
                avion.velocidad_actual = 200
                print(f" (!) Avion {avion.id} inicia maniobra de regreso.")
            else:
                # Puede reducir de forma segura.
                avion.estado = "AJUSTANDO_VELOCIDAD"
                avion.velocidad_actual = velocidad_minima
        else:
            # Hay espacio suficiente. Va a máxima velocidad.
            avion.estado = "APROXIMANDO"
            avion.velocidad_actual = calcular_velocidad_maxima_permitida(avion.distancia_a_aep)



def actualizar_estados_y_velocidades_politica_2(aviones, resultados=None, minuto=None, warmup=0):
    """Actualiza el estado y la velocidad de cada avión según las reglas."""
    aviones.sort(key=lambda avion: avion.distancia_a_aep)
    for i, avion in enumerate(aviones):
        if avion.estado == "REGRESANDO":
            gestionar_logica_regreso(avion, aviones, resultados, minuto, warmup)

        else: # APROXIMANDO o AJUSTANDO_VELOCIDAD
            avion_de_adelante = aviones[i-1] if i > 0 and aviones[i-1].estado != "REGRESANDO" else None
            gestionar_logica_aproximacion_politica_2(avion, avion_de_adelante)

        if resultados is not None and avion.estado == "AJUSTANDO_VELOCIDAD" and minuto is not None and minuto >= warmup:
            resultados['congestion_events'] += 1


In [ ]:
def simular_una_corrida(lambda_val, tiempo_total=1080, warmup=0):
    """
    Ejecuta una corrida completa de la simulación para un lambda dado
    y devuelve un diccionario con los resultados.
    """
    # --- Parámetros y Almacenamiento ---
    aviones_activos = []
    aviones_finalizados = []
    sistema_migraciones = SistemaMigraciones(prob_auto=0.66)  
    next_avion_id = 1
    TIEMPO_IDEAL_VIAJE = 23.4  # Nuestro baseline en minutos
    

    # --- Contadores para ESTA corrida específica ---
    resultados_de_la_corrida = {
        'total_delay_min': 0.0,
        'congestion_events': 0,
        'diversions': 0,
        'landed_planes': 0,
        'inbound_planes': 0,
        'total_planes': 0,
        'active_planes':0,
        'returning_planes':0,
        'queue_size':0, 
        'queue_size_dom':0,
        'queue_size_int':0,
        'wait_time_avg':0.0,
        'wait_time_dom':0.0,
        'wait_time_int':0.0
    }

    historial_metricas = {
        'minuto': [],
        'delay_acum': [],
        'congestion_events': [],
        'diversions': [],
        'landed_planes': [],
        'delay_avg': [],
        'inbound_planes': [],
        'active_planes':[],
        'returning_planes':[],
        'queue_size':[],
        'queue_size_dom':[],
        'queue_size_int':[]
    }

    # --- Bucle de un día de simulación ---
    for minuto in range(tiempo_total):
        # Generar arribos (ahora pasa el minuto de creación)
        if random.random() < lambda_val:
            # Generamos el modelo de avión con su cantidad de pasajeros
            modelo = np.random.choice(list(MODELOS.keys()))
            capacidad = MODELOS[modelo]

            # Generamos cantidad de pasajeros como lognormal centrada en 0.95 * capacidad
            pasajeros_a_bordo = min(int(np.random.lognormal(mean=np.log(0.95 * capacidad), sigma=5)), capacidad)

            # Nacional o internacional
            tipo = np.random.choice(["domestico", "internacional"], p=[0.74, 0.26])
            nuevo_avion = Avion(next_avion_id, minuto, pasajeros_a_bordo, tipo)
            aviones_activos.append(nuevo_avion)
            next_avion_id += 1
            if minuto >= warmup:
                resultados_de_la_corrida['total_planes'] += 1

        # Actualizar estados y contar congestiones
        actualizar_estados_y_velocidades_politica_2(aviones_activos, resultados_de_la_corrida, minuto=minuto, warmup=warmup)

        # Mover aviones
        mover_aviones(aviones_activos)

        # Gestionar aviones que terminaron (y contar retrasos, desvíos, aterrizajes)
        activos_actualizados, finalizados_ahora = gestionar_aviones_finalizados(
            minuto, aviones_activos, resultados_de_la_corrida, TIEMPO_IDEAL_VIAJE, warmup=warmup
        )
        aviones_activos = activos_actualizados
        aviones_finalizados.extend(finalizados_ahora)

        # Bajar pasajeros a la cola de migraciones
        for avion in finalizados_ahora:
            if avion.estado == "ATERRIZADO":
                sistema_migraciones.add_arrivals(avion.pasajeros, minuto, avion.tipo)

        # Actualizar sistema de migraciones una vez por minuto (asignaciones a puestos)
        migr_stats = sistema_migraciones.step(minuto)

        # Mantener métricas en resultados/historial (igual que antes)
        cola_migraciones = migr_stats['en_cola']
        if minuto >= warmup:
            historial_metricas['queue_size'].append(cola_migraciones)
            historial_metricas['queue_size_dom'].append(migr_stats['cola_domesticos'])
            historial_metricas['queue_size_int'].append(migr_stats['cola_internacionales'])
        else:
            historial_metricas['queue_size'].append(0)
            historial_metricas['queue_size_dom'].append(0)
            historial_metricas['queue_size_int'].append(0)

        # Guardar métricas del minuto (solo si pasó el warm-up)
        if minuto >= warmup:
            for avion in aviones_activos:
                if avion.estado == "APROXIMANDO" or avion.estado == "AJUSTANDO_VELOCIDAD":
                    resultados_de_la_corrida['inbound_planes'] += 1
                elif avion.estado == "REGRESANDO":
                    resultados_de_la_corrida['returning_planes'] += 1
            
            for avion in finalizados_ahora:
                if avion.estado == "DESVIADO":
                    resultados_de_la_corrida['diversions'] += 1
                elif avion.estado == "ATERRIZADO":
                    resultados_de_la_corrida['landed_planes'] += 1

        historial_metricas['minuto'].append(minuto)
        historial_metricas['delay_acum'].append(resultados_de_la_corrida['total_delay_min'])
        historial_metricas['congestion_events'].append(resultados_de_la_corrida['congestion_events'])
        historial_metricas['diversions'].append(resultados_de_la_corrida['diversions'])
        historial_metricas['landed_planes'].append(resultados_de_la_corrida['landed_planes'])
        landed = resultados_de_la_corrida['landed_planes']
        if landed > 0:
            avg_delay = resultados_de_la_corrida['total_delay_min'] / landed
        else:
            avg_delay = 0.0
        historial_metricas['delay_avg'].append(avg_delay)
        historial_metricas['inbound_planes'].append(resultados_de_la_corrida['inbound_planes'])
        historial_metricas['active_planes'].append(len(aviones_activos))
        historial_metricas['returning_planes'].append(resultados_de_la_corrida['returning_planes'])
    
    # --- Métricas finales ---
    resultados_de_la_corrida['active_planes'] = sum(historial_metricas['active_planes'])
    resultados_de_la_corrida['queue_size'] = np.mean(historial_metricas['queue_size'])
    resultados_de_la_corrida['queue_size_dom'] = np.mean(historial_metricas['queue_size_dom'])
    resultados_de_la_corrida['queue_size_int'] = np.mean(historial_metricas['queue_size_int'])
    resultados_de_la_corrida['wait_time_avg'] = sistema_migraciones.promedio_espera()
    resultados_de_la_corrida['wait_time_dom'] = sistema_migraciones.promedio_espera("domestico")
    resultados_de_la_corrida['wait_time_int'] = sistema_migraciones.promedio_espera("internacional")

    return resultados_de_la_corrida, pd.DataFrame(historial_metricas)

def ejecutar_experimentos():
    """
    Función principal que ejecuta el bucle experimental para varios lambdas.
    """
    # --- Configuración del Experimento ---
    lambdas_a_probar = [0.02, 0.1, 0.2, 0.5, 1.0]
    N_REPETICIONES = 50 # Número de veces que se repite la simulación para cada lambda
    WARMUP_MIN = 60 

    resultados_finales = []
    historiales = []
    print("--- Iniciando Bucle Experimental ---")
    for lambda_val in lambdas_a_probar:
        print(f"\n--- Probando con λ = {lambda_val:.4f} ---")
        for i in range(N_REPETICIONES):
            resultado_run, df_hist = simular_una_corrida(lambda_val, warmup=WARMUP_MIN)
            resultado_run['lambda'] = lambda_val
            resultados_finales.append(resultado_run)
            df_hist['lambda'] = lambda_val
            df_hist['rep'] = i
            historiales.append(df_hist)
            print(".", end="")
    print("\n\n--- Bucle Experimental Finalizado ---")
    df_resultados = pd.DataFrame(resultados_finales)
    df_historiales = pd.concat(historiales, ignore_index=True)
    return df_resultados, df_historiales

if __name__ == '__main__':
    np.random.seed(42)
    random.seed(42)
    df_final_resultados_politica_2, df_hist = ejecutar_experimentos()

In [ ]:
df_estadisticas_politica_2 = df_final_resultados_politica_2.groupby('lambda').agg(
    n_simulaciones=('lambda', 'count'),
    total_aviones_promedio=('total_planes', 'mean'),
    cola_migraciones_promedio=('queue_size', 'mean'),
    cola_migraciones_std=('queue_size', 'std'),
    tiempo_espera_promedio=('wait_time_avg', 'mean'),
    tiempo_espera_std=('wait_time_avg', 'std'),
    tiempo_espera_domesticos_promedio=('wait_time_dom', 'mean'),
    tiempo_espera_internacionales_promedio=('wait_time_int', 'mean'),
    tiempo_espera_internacionales_std=('wait_time_int', 'std'),
    tiempo_espera_domesticos_std=('wait_time_dom', 'std'),
    cola_migraciones_domesticos_promedio=('queue_size_dom', 'mean'),
    cola_migraciones_domesticos_std=('queue_size_dom', 'std'),
    cola_migraciones_internacionales_promedio=('queue_size_int', 'mean'),
    cola_migraciones_internacionales_std=('queue_size_int', 'std')
)
# 2. Calcular el Error Estándar de la Media (nuestro error de estimación)
df_estadisticas_politica_2['tiempo_espera_error'] = 1.96 * df_estadisticas_politica_2['tiempo_espera_std'] / np.sqrt(df_estadisticas_politica_2['n_simulaciones'])
df_estadisticas_politica_2['tiempo_espera_domesticos_error'] = 1.96 * df_estadisticas_politica_2['tiempo_espera_domesticos_std'] / np.sqrt(df_estadisticas_politica_2['n_simulaciones'])
df_estadisticas_politica_2['tiempo_espera_internacionales_error'] = 1.96 * df_estadisticas_politica_2['tiempo_espera_internacionales_std'] / np.sqrt(df_estadisticas_politica_2['n_simulaciones'])
df_estadisticas_politica_2['cola_migraciones_domesticos_error'] = 1.96 * df_estadisticas_politica_2['cola_migraciones_domesticos_std'] / np.sqrt(df_estadisticas_politica_2['n_simulaciones'])
df_estadisticas_politica_2['cola_migraciones_internacionales_error'] = 1.96 * df_estadisticas_politica_2['cola_migraciones_internacionales_std'] / np.sqrt(df_estadisticas_politica_2['n_simulaciones'])
df_estadisticas_politica_2['cola_migraciones_error'] = 1.96 * df_estadisticas_politica_2['cola_migraciones_std'] / np.sqrt(df_estadisticas_politica_2['n_simulaciones'])

# Eliminar atraso y stds
df_estadisticas_politica_2 = df_estadisticas_politica_2.drop(columns=['cola_migraciones_std', 'tiempo_espera_std',
                                                 'tiempo_espera_domesticos_std', 'tiempo_espera_internacionales_std',
                                                 'cola_migraciones_domesticos_std', 'cola_migraciones_internacionales_std'])

# Mostramos la tabla final de resultados
print("--- Tabla de Estadísticas por Valor de λ ---")
display(df_estadisticas_politica_2)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True)
fig.suptitle('Impacto de implementación de política 2 en Migraciones', fontsize=18, weight='bold')

# --- Cola Domésticos ---
ax1 = axes[0,0]
ax1.plot(df_estadisticas.index, df_estadisticas['cola_migraciones_domesticos_promedio'], 
         marker='o', linestyle='--', color='royalblue', label='Día Normal')
ax1.plot(df_estadisticas_politica_2.index, df_estadisticas_politica_2['cola_migraciones_domesticos_promedio'], 
         marker='o', linestyle='-', color='deepskyblue', label='Con Política')
ax1.set_title('Cola Migraciones - Domésticos', fontsize=14)
ax1.set_ylabel('Tamaño Promedio de la Cola')
ax1.set_xlabel('Tasa de Arribo (λ)', fontsize=12)
ax1.legend()

# --- Cola Internacionales ---
ax2 = axes[0,1]
ax2.plot(df_estadisticas.index, df_estadisticas['cola_migraciones_internacionales_promedio'], 
         marker='s', linestyle='--', color='firebrick', label='Día Normal')
ax2.plot(df_estadisticas_politica_2.index, df_estadisticas_politica_2['cola_migraciones_internacionales_promedio'], 
         marker='s', linestyle='-', color='salmon', label='Con Política')
ax2.set_title('Cola Migraciones - Internacionales', fontsize=14)
ax2.set_ylabel('Tamaño Promedio de la Cola')
ax2.set_xlabel('Tasa de Arribo (λ)', fontsize=12)
ax2.legend()

# --- Espera Domésticos ---
ax3 = axes[1,0]
ax3.plot(df_estadisticas.index, df_estadisticas['tiempo_espera_domesticos_promedio'], 
         marker='o', linestyle='--', color='royalblue', label='Día Normal')
ax3.plot(df_estadisticas_politica_2.index, df_estadisticas_politica_2['tiempo_espera_domesticos_promedio'], 
         marker='o', linestyle='-', color='deepskyblue', label='Con Política')
ax3.set_title('Tiempo de Espera - Domésticos', fontsize=14)
ax3.set_ylabel('Minutos de Espera')
ax3.set_xlabel('Tasa de Arribo (λ)', fontsize=12)
ax3.legend()

# --- Espera Internacionales ---
ax4 = axes[1,1]
ax4.plot(df_estadisticas.index, df_estadisticas['tiempo_espera_internacionales_promedio'], 
         marker='s', linestyle='--', color='firebrick', label='Día Normal')
ax4.plot(df_estadisticas_politica_2.index, df_estadisticas_politica_2['tiempo_espera_internacionales_promedio'], 
         marker='s', linestyle='-', color='salmon', label='Con Política')
ax4.set_title('Tiempo de Espera - Internacionales', fontsize=14)
ax4.set_ylabel('Minutos de Espera')
ax4.set_xlabel('Tasa de Arribo (λ)', fontsize=12)
ax4.legend()

plt.tight_layout(rect=[0, 0, 1, 0.96])  # deja espacio para el título general
plt.show()


## Política 3: vía rápida

In [ ]:
def gestionar_logica_aproximacion_via_rapida(avion, avion_de_adelante):
    """
    Política "Vía Rápida" (o "Anti-Embudo").
    Si la fila de adelante está lenta, el avión actual prefiere regresar antes que frenar.
    """
    if avion_de_adelante is None:
        # No hay nadie adelante, volar a máxima velocidad.
        avion.estado = "APROXIMANDO"
        avion.velocidad_actual = calcular_velocidad_maxima_permitida(avion.distancia_a_aep)
        return

    tiempo_sep = calcular_separacion_en_tiempo(avion, avion_de_adelante)

    if tiempo_sep < 4:
        # --- LÓGICA CORREGIDA DE LA POLÍTICA ---
        velocidad_max_adelante = calcular_velocidad_maxima_permitida(avion_de_adelante.distancia_a_aep)
        
        # ¿El avión de adelante es un "obstáculo" lento?
        if avion_de_adelante.velocidad_actual < (velocidad_max_adelante - 15):
            # SÍ ES UN OBSTÁCULO: El avión actual inicia una maniobra de regreso
            avion_de_adelante.estado = "REGRESANDO"
            avion_de_adelante.velocidad_actual = 200
            avion.estado = "APROXIMANDO"
            avion.velocidad_actual = calcular_velocidad_maxima_permitida(avion.distancia_a_aep)
            
        else:
            # NO ES UN OBSTÁCULO: El avión actual intenta frenar (lógica original)
            velocidad_req = avion_de_adelante.velocidad_actual - 20
            if velocidad_req < calcular_velocidad_minima_permitida(avion.distancia_a_aep):
                avion.estado = "REGRESANDO"
                avion.velocidad_actual = 200
            else:
                avion.estado = "AJUSTANDO_VELOCIDAD"
                avion.velocidad_actual = velocidad_req
    else:
        # Hay espacio suficiente, volar a máxima velocidad.
        avion.estado = "APROXIMANDO"
        avion.velocidad_actual = calcular_velocidad_maxima_permitida(avion.distancia_a_aep)

def actualizar_estados_y_velocidades_politica_3(aviones, resultados=None, minuto=None, warmup=0):
    """Actualiza el estado y la velocidad de cada avión según las reglas."""
    aviones.sort(key=lambda avion: avion.distancia_a_aep)
    for i, avion in enumerate(aviones):
        if avion.estado == "REGRESANDO":
            gestionar_logica_regreso(avion, aviones, resultados, minuto, warmup)

        else: # APROXIMANDO o AJUSTANDO_VELOCIDAD
            avion_de_adelante = aviones[i-1] if i > 0 and aviones[i-1].estado != "REGRESANDO" else None
            gestionar_logica_aproximacion_via_rapida(avion, avion_de_adelante)

        if resultados is not None and avion.estado == "AJUSTANDO_VELOCIDAD" and minuto is not None and minuto >= warmup:
            resultados['congestion_events'] += 1

In [ ]:
def simular_una_corrida(lambda_val, tiempo_total=1080, warmup=0):
    """
    Ejecuta una corrida completa de la simulación para un lambda dado
    y devuelve un diccionario con los resultados.
    """
    # --- Parámetros y Almacenamiento ---
    aviones_activos = []
    aviones_finalizados = []
    next_avion_id = 1
    TIEMPO_IDEAL_VIAJE = 23.4  # Nuestro baseline en minutos
    sistema_migraciones = SistemaMigraciones(prob_auto=0.66) 
    

    # --- Contadores para ESTA corrida específica ---
    resultados_de_la_corrida = {
        'total_delay_min': 0.0,
        'congestion_events': 0,
        'diversions': 0,
        'landed_planes': 0,
        'inbound_planes': 0,
        'total_planes': 0,
        'active_planes':0,
        'returning_planes':0,
        'queue_size':0, 
        'queue_size_dom':0,
        'queue_size_int':0,
        'wait_time_avg':0.0,
        'wait_time_dom':0.0,
        'wait_time_int':0.0
    }

    historial_metricas = {
        'minuto': [],
        'delay_acum': [],
        'congestion_events': [],
        'diversions': [],
        'landed_planes': [],
        'delay_avg': [],
        'inbound_planes': [],
        'active_planes':[],
        'returning_planes':[],
        'queue_size':[],
        'queue_size_dom':[],
        'queue_size_int':[]
    }

    # --- Bucle de un día de simulación ---
    for minuto in range(tiempo_total):
        # Generar arribos (ahora pasa el minuto de creación)
        if random.random() < lambda_val:
            # Generamos el modelo de avión con su cantidad de pasajeros
            modelo = np.random.choice(list(MODELOS.keys()))
            capacidad = MODELOS[modelo]

            # Generamos cantidad de pasajeros como lognormal centrada en 0.95 * capacidad
            pasajeros_a_bordo = min(int(np.random.lognormal(mean=np.log(0.95 * capacidad), sigma=5)), capacidad)

            # Nacional o internacional
            tipo = np.random.choice(["domestico", "internacional"], p=[0.74, 0.26])
            nuevo_avion = Avion(next_avion_id, minuto, pasajeros_a_bordo, tipo)
            aviones_activos.append(nuevo_avion)
            next_avion_id += 1
            if minuto >= warmup:
                resultados_de_la_corrida['total_planes'] += 1

        # Actualizar estados y contar congestiones
        actualizar_estados_y_velocidades_politica_3(aviones_activos, resultados_de_la_corrida, minuto=minuto, warmup=warmup)

        # Mover aviones
        mover_aviones(aviones_activos)

        # Gestionar aviones que terminaron (y contar retrasos, desvíos, aterrizajes)
        activos_actualizados, finalizados_ahora = gestionar_aviones_finalizados(
            minuto, aviones_activos, resultados_de_la_corrida, TIEMPO_IDEAL_VIAJE, warmup=warmup
        )
        aviones_activos = activos_actualizados
        aviones_finalizados.extend(finalizados_ahora)

        # Bajar pasajeros a la cola de migraciones
        for avion in finalizados_ahora:
            if avion.estado == "ATERRIZADO":
                sistema_migraciones.add_arrivals(avion.pasajeros, minuto, avion.tipo)

        # Actualizar sistema de migraciones una vez por minuto (asignaciones a puestos)
        migr_stats = sistema_migraciones.step(minuto)

        # Mantener métricas en resultados/historial (igual que antes)
        cola_migraciones = migr_stats['en_cola']
        if minuto >= warmup:
            historial_metricas['queue_size'].append(cola_migraciones)
            historial_metricas['queue_size_dom'].append(migr_stats['cola_domesticos'])
            historial_metricas['queue_size_int'].append(migr_stats['cola_internacionales'])
        else:
            historial_metricas['queue_size'].append(0)
            historial_metricas['queue_size_dom'].append(0)
            historial_metricas['queue_size_int'].append(0)

        # Guardar métricas del minuto (solo si pasó el warm-up)
        if minuto >= warmup:
            for avion in aviones_activos:
                if avion.estado == "APROXIMANDO" or avion.estado == "AJUSTANDO_VELOCIDAD":
                    resultados_de_la_corrida['inbound_planes'] += 1
                elif avion.estado == "REGRESANDO":
                    resultados_de_la_corrida['returning_planes'] += 1
            
            for avion in finalizados_ahora:
                if avion.estado == "DESVIADO":
                    resultados_de_la_corrida['diversions'] += 1
                elif avion.estado == "ATERRIZADO":
                    resultados_de_la_corrida['landed_planes'] += 1

        historial_metricas['minuto'].append(minuto)
        historial_metricas['delay_acum'].append(resultados_de_la_corrida['total_delay_min'])
        historial_metricas['congestion_events'].append(resultados_de_la_corrida['congestion_events'])
        historial_metricas['diversions'].append(resultados_de_la_corrida['diversions'])
        historial_metricas['landed_planes'].append(resultados_de_la_corrida['landed_planes'])
        landed = resultados_de_la_corrida['landed_planes']
        if landed > 0:
            avg_delay = resultados_de_la_corrida['total_delay_min'] / landed
        else:
            avg_delay = 0.0
        historial_metricas['delay_avg'].append(avg_delay)
        historial_metricas['inbound_planes'].append(resultados_de_la_corrida['inbound_planes'])
        historial_metricas['active_planes'].append(len(aviones_activos))
        historial_metricas['returning_planes'].append(resultados_de_la_corrida['returning_planes'])
    
    resultados_de_la_corrida['active_planes'] = sum(historial_metricas['active_planes'])
    resultados_de_la_corrida['queue_size'] = np.mean(historial_metricas['queue_size'])
    resultados_de_la_corrida['queue_size_dom'] = np.mean(historial_metricas['queue_size_dom'])
    resultados_de_la_corrida['queue_size_int'] = np.mean(historial_metricas['queue_size_int'])
    resultados_de_la_corrida['wait_time_avg'] = sistema_migraciones.promedio_espera()
    resultados_de_la_corrida['wait_time_dom'] = sistema_migraciones.promedio_espera("domestico")
    resultados_de_la_corrida['wait_time_int'] = sistema_migraciones.promedio_espera("internacional")

    return resultados_de_la_corrida, pd.DataFrame(historial_metricas)

def ejecutar_experimentos():
    """
    Función principal que ejecuta el bucle experimental para varios lambdas.
    """
    # --- Configuración del Experimento ---
    lambdas_a_probar = [0.02, 0.1, 0.2, 0.5, 1.0]
    N_REPETICIONES = 50 # Número de veces que se repite la simulación para cada lambda
    WARMUP_MIN = 60 

    resultados_finales = []
    historiales = []
    print("--- Iniciando Bucle Experimental ---")
    for lambda_val in lambdas_a_probar:
        print(f"\n--- Probando con λ = {lambda_val:.4f} ---")
        for i in range(N_REPETICIONES):
            resultado_run, df_hist = simular_una_corrida(lambda_val, warmup=WARMUP_MIN)
            resultado_run['lambda'] = lambda_val
            resultados_finales.append(resultado_run)
            df_hist['lambda'] = lambda_val
            df_hist['rep'] = i
            historiales.append(df_hist)
            print(".", end="")
    print("\n\n--- Bucle Experimental Finalizado ---")
    df_resultados = pd.DataFrame(resultados_finales)
    df_historiales = pd.concat(historiales, ignore_index=True)
    return df_resultados, df_historiales

if __name__ == '__main__':
    np.random.seed(42)
    random.seed(42)
    df_final_resultados_politica_3, df_hist = ejecutar_experimentos()

In [ ]:
df_estadisticas_politica_3 = df_final_resultados_politica_3.groupby('lambda').agg(
    n_simulaciones=('lambda', 'count'),
    total_aviones_promedio=('total_planes', 'mean'),
    cola_migraciones_promedio=('queue_size', 'mean'),
    cola_migraciones_std=('queue_size', 'std'),
    tiempo_espera_promedio=('wait_time_avg', 'mean'),
    tiempo_espera_std=('wait_time_avg', 'std'),
    tiempo_espera_domesticos_promedio=('wait_time_dom', 'mean'),
    tiempo_espera_internacionales_promedio=('wait_time_int', 'mean'),
    tiempo_espera_internacionales_std=('wait_time_int', 'std'),
    tiempo_espera_domesticos_std=('wait_time_dom', 'std'),
    cola_migraciones_domesticos_promedio=('queue_size_dom', 'mean'),
    cola_migraciones_domesticos_std=('queue_size_dom', 'std'),
    cola_migraciones_internacionales_promedio=('queue_size_int', 'mean'),
    cola_migraciones_internacionales_std=('queue_size_int', 'std')
)
# 2. Calcular el Error Estándar de la Media (nuestro error de estimación)
df_estadisticas_politica_3['tiempo_espera_error'] = 1.96 * df_estadisticas_politica_3['tiempo_espera_std'] / np.sqrt(df_estadisticas_politica_3['n_simulaciones'])
df_estadisticas_politica_3['tiempo_espera_domesticos_error'] = 1.96 * df_estadisticas_politica_3['tiempo_espera_domesticos_std'] / np.sqrt(df_estadisticas_politica_3['n_simulaciones'])
df_estadisticas_politica_3['tiempo_espera_internacionales_error'] = 1.96 * df_estadisticas_politica_3['tiempo_espera_internacionales_std'] / np.sqrt(df_estadisticas_politica_3['n_simulaciones'])
df_estadisticas_politica_3['cola_migraciones_domesticos_error'] = 1.96 * df_estadisticas_politica_3['cola_migraciones_domesticos_std'] / np.sqrt(df_estadisticas_politica_3['n_simulaciones'])
df_estadisticas_politica_3['cola_migraciones_internacionales_error'] = 1.96 * df_estadisticas_politica_3['cola_migraciones_internacionales_std'] / np.sqrt(df_estadisticas_politica_3['n_simulaciones'])
df_estadisticas_politica_3['cola_migraciones_error'] = 1.96 * df_estadisticas_politica_3['cola_migraciones_std'] / np.sqrt(df_estadisticas_politica_3['n_simulaciones'])

# Eliminar atraso y stds
df_estadisticas_politica_3 = df_estadisticas_politica_3.drop(columns=['cola_migraciones_std', 'tiempo_espera_std',
                                                 'tiempo_espera_domesticos_std', 'tiempo_espera_internacionales_std',
                                                 'cola_migraciones_domesticos_std', 'cola_migraciones_internacionales_std'])

# Mostramos la tabla final de resultados
print("--- Tabla de Estadísticas por Valor de λ ---")
display(df_estadisticas_politica_3)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True)
fig.suptitle('Impacto de implementación de política 3 en Migraciones', fontsize=18, weight='bold')

# --- Cola Domésticos ---
ax1 = axes[0,0]
ax1.plot(df_estadisticas.index, df_estadisticas['cola_migraciones_domesticos_promedio'], 
         marker='o', linestyle='--', color='royalblue', label='Día Normal')
ax1.plot(df_estadisticas_politica_3.index, df_estadisticas_politica_3['cola_migraciones_domesticos_promedio'], 
         marker='o', linestyle='-', color='deepskyblue', label='Con Política')
ax1.set_title('Cola Migraciones - Domésticos', fontsize=14)
ax1.set_ylabel('Tamaño Promedio de la Cola')
ax1.set_xlabel('Tasa de Arribo (λ)', fontsize=12)
ax1.legend()

# --- Cola Internacionales ---
ax2 = axes[0,1]
ax2.plot(df_estadisticas.index, df_estadisticas['cola_migraciones_internacionales_promedio'], 
         marker='s', linestyle='--', color='firebrick', label='Día Normal')
ax2.plot(df_estadisticas_politica_3.index, df_estadisticas_politica_3['cola_migraciones_internacionales_promedio'], 
         marker='s', linestyle='-', color='salmon', label='Con Política')
ax2.set_title('Cola Migraciones - Internacionales', fontsize=14)
ax2.set_ylabel('Tamaño Promedio de la Cola')
ax2.set_xlabel('Tasa de Arribo (λ)', fontsize=12)
ax2.legend()

# --- Espera Domésticos ---
ax3 = axes[1,0]
ax3.plot(df_estadisticas.index, df_estadisticas['tiempo_espera_domesticos_promedio'], 
         marker='o', linestyle='--', color='royalblue', label='Día Normal')
ax3.plot(df_estadisticas_politica_3.index, df_estadisticas_politica_3['tiempo_espera_domesticos_promedio'], 
         marker='o', linestyle='-', color='deepskyblue', label='Con Política')
ax3.set_title('Tiempo de Espera - Domésticos', fontsize=14)
ax3.set_ylabel('Minutos de Espera')
ax3.set_xlabel('Tasa de Arribo (λ)', fontsize=12)
ax3.legend()

# --- Espera Internacionales ---
ax4 = axes[1,1]
ax4.plot(df_estadisticas.index, df_estadisticas['tiempo_espera_internacionales_promedio'], 
         marker='s', linestyle='--', color='firebrick', label='Día Normal')
ax4.plot(df_estadisticas_politica_3.index, df_estadisticas_politica_3['tiempo_espera_internacionales_promedio'], 
         marker='s', linestyle='-', color='salmon', label='Con Política')
ax4.set_title('Tiempo de Espera - Internacionales', fontsize=14)
ax4.set_ylabel('Minutos de Espera')
ax4.set_xlabel('Tasa de Arribo (λ)', fontsize=12)
ax4.legend()

plt.tight_layout(rect=[0, 0, 1, 0.96])  # deja espacio para el título general
plt.show()


## Política 4: reducción más fuerte (10 kt)

In [ ]:
def gestionar_logica_aproximacion_politica_4(avion, avion_de_adelante):
    """
    Política "Vía Rápida" (o "Anti-Embudo").
    Si la fila de adelante está lenta, el avión actual prefiere regresar antes que frenar.
    """
    if avion_de_adelante is None:
        # No hay nadie adelante, volar a máxima velocidad.
        avion.estado = "APROXIMANDO"
        avion.velocidad_actual = calcular_velocidad_maxima_permitida(avion.distancia_a_aep)
        return

    tiempo_sep = calcular_separacion_en_tiempo(avion, avion_de_adelante)

    if tiempo_sep < 4:

        velocidad_req = avion.velocidad_actual - 10
        if velocidad_req < calcular_velocidad_minima_permitida(avion.distancia_a_aep):
            avion.estado = "REGRESANDO"
            avion.velocidad_actual = 200
        else:
            avion.estado = "AJUSTANDO_VELOCIDAD"
            avion.velocidad_actual = velocidad_req
    else:
        # Hay espacio suficiente, volar a máxima velocidad.
        avion.estado = "APROXIMANDO"
        avion.velocidad_actual = calcular_velocidad_maxima_permitida(avion.distancia_a_aep)



def actualizar_estados_y_velocidades_politica_4(aviones, resultados=None, minuto=None, warmup=0):
    """Actualiza el estado y la velocidad de cada avión según las reglas."""
    aviones.sort(key=lambda avion: avion.distancia_a_aep)
    for i, avion in enumerate(aviones):
        if avion.estado == "REGRESANDO":
            gestionar_logica_regreso(avion, aviones, resultados, minuto, warmup)

        else: # APROXIMANDO o AJUSTANDO_VELOCIDAD
            avion_de_adelante = aviones[i-1] if i > 0 and aviones[i-1].estado != "REGRESANDO" else None
            gestionar_logica_aproximacion_politica_4(avion, avion_de_adelante)

        if resultados is not None and avion.estado == "AJUSTANDO_VELOCIDAD" and minuto is not None and minuto >= warmup:
            resultados['congestion_events'] += 1

In [ ]:
def simular_una_corrida(lambda_val, tiempo_total=1080, warmup=0):
    """
    Ejecuta una corrida completa de la simulación para un lambda dado
    y devuelve un diccionario con los resultados.
    """
    # --- Parámetros y Almacenamiento ---
    aviones_activos = []
    aviones_finalizados = []
    next_avion_id = 1
    TIEMPO_IDEAL_VIAJE = 23.4  # Nuestro baseline en minutos
    sistema_migraciones = SistemaMigraciones(prob_auto=0.66) 
    

    # --- Contadores para ESTA corrida específica ---
    resultados_de_la_corrida = {
        'total_delay_min': 0.0,
        'congestion_events': 0,
        'diversions': 0,
        'landed_planes': 0,
        'inbound_planes': 0,
        'total_planes': 0,
        'active_planes':0,
        'returning_planes':0,
        'queue_size':0, 
        'queue_size_dom':0,
        'queue_size_int':0,
        'wait_time_avg':0.0,
        'wait_time_dom':0.0,
        'wait_time_int':0.0
    }

    historial_metricas = {
        'minuto': [],
        'delay_acum': [],
        'congestion_events': [],
        'diversions': [],
        'landed_planes': [],
        'delay_avg': [],
        'inbound_planes': [],
        'active_planes':[],
        'returning_planes':[],
        'queue_size':[],
        'queue_size_dom':[],
        'queue_size_int':[]
    }


    # --- Bucle de un día de simulación ---
    for minuto in range(tiempo_total):
        # Generar arribos (ahora pasa el minuto de creación)
        if random.random() < lambda_val:
            # Generamos el modelo de avión con su cantidad de pasajeros
            modelo = np.random.choice(list(MODELOS.keys()))
            capacidad = MODELOS[modelo]

            # Generamos cantidad de pasajeros como lognormal centrada en 0.95 * capacidad
            pasajeros_a_bordo = min(int(np.random.lognormal(mean=np.log(0.95 * capacidad), sigma=5)), capacidad)

            # Nacional o internacional
            tipo = np.random.choice(["domestico", "internacional"], p=[0.74, 0.26])
            nuevo_avion = Avion(next_avion_id, minuto, pasajeros_a_bordo, tipo)
            aviones_activos.append(nuevo_avion)
            next_avion_id += 1
            if minuto >= warmup:
                resultados_de_la_corrida['total_planes'] += 1
        # Actualizar estados y contar congestiones
        actualizar_estados_y_velocidades_politica_4(aviones_activos, resultados_de_la_corrida, minuto=minuto, warmup=warmup)

        # Mover aviones
        mover_aviones(aviones_activos)

        # Gestionar aviones que terminaron (y contar retrasos, desvíos, aterrizajes)
        activos_actualizados, finalizados_ahora = gestionar_aviones_finalizados(
            minuto, aviones_activos, resultados_de_la_corrida, TIEMPO_IDEAL_VIAJE, warmup=warmup
        )
        aviones_activos = activos_actualizados
        aviones_finalizados.extend(finalizados_ahora)

        # Bajar pasajeros a la cola de migraciones
        for avion in finalizados_ahora:
            if avion.estado == "ATERRIZADO":
                sistema_migraciones.add_arrivals(avion.pasajeros, minuto, avion.tipo)

        # Actualizar sistema de migraciones una vez por minuto (asignaciones a puestos)
        migr_stats = sistema_migraciones.step(minuto)

        # Mantener métricas en resultados/historial (igual que antes)
        cola_migraciones = migr_stats['en_cola']
        if minuto >= warmup:
            historial_metricas['queue_size'].append(cola_migraciones)
            historial_metricas['queue_size_dom'].append(migr_stats['cola_domesticos'])
            historial_metricas['queue_size_int'].append(migr_stats['cola_internacionales'])
        else:
            historial_metricas['queue_size'].append(0)
            historial_metricas['queue_size_dom'].append(0)
            historial_metricas['queue_size_int'].append(0)


        # Guardar métricas del minuto (solo si pasó el warm-up)
        if minuto >= warmup:
            for avion in aviones_activos:
                if avion.estado == "APROXIMANDO" or avion.estado == "AJUSTANDO_VELOCIDAD":
                    resultados_de_la_corrida['inbound_planes'] += 1
                elif avion.estado == "REGRESANDO":
                    resultados_de_la_corrida['returning_planes'] += 1
            
            for avion in finalizados_ahora:
                if avion.estado == "DESVIADO":
                    resultados_de_la_corrida['diversions'] += 1
                elif avion.estado == "ATERRIZADO":
                    resultados_de_la_corrida['landed_planes'] += 1

        historial_metricas['minuto'].append(minuto)
        historial_metricas['delay_acum'].append(resultados_de_la_corrida['total_delay_min'])
        historial_metricas['congestion_events'].append(resultados_de_la_corrida['congestion_events'])
        historial_metricas['diversions'].append(resultados_de_la_corrida['diversions'])
        historial_metricas['landed_planes'].append(resultados_de_la_corrida['landed_planes'])
        landed = resultados_de_la_corrida['landed_planes']
        if landed > 0:
            avg_delay = resultados_de_la_corrida['total_delay_min'] / landed
        else:
            avg_delay = 0.0
        historial_metricas['delay_avg'].append(avg_delay)
        historial_metricas['inbound_planes'].append(resultados_de_la_corrida['inbound_planes'])
        historial_metricas['active_planes'].append(len(aviones_activos))
        historial_metricas['returning_planes'].append(resultados_de_la_corrida['returning_planes'])
    
    resultados_de_la_corrida['active_planes'] = sum(historial_metricas['active_planes'])
    resultados_de_la_corrida['queue_size'] = np.mean(historial_metricas['queue_size'])
    resultados_de_la_corrida['queue_size_dom'] = np.mean(historial_metricas['queue_size_dom'])
    resultados_de_la_corrida['queue_size_int'] = np.mean(historial_metricas['queue_size_int'])
    resultados_de_la_corrida['wait_time_avg'] = sistema_migraciones.promedio_espera()
    resultados_de_la_corrida['wait_time_dom'] = sistema_migraciones.promedio_espera("domestico")
    resultados_de_la_corrida['wait_time_int'] = sistema_migraciones.promedio_espera("internacional")

    return resultados_de_la_corrida, pd.DataFrame(historial_metricas)

def ejecutar_experimentos():
    """
    Función principal que ejecuta el bucle experimental para varios lambdas.
    """
    # --- Configuración del Experimento ---
    lambdas_a_probar = [0.02, 0.1, 0.2, 0.5, 1.0]
    N_REPETICIONES = 50 # Número de veces que se repite la simulación para cada lambda
    WARMUP_MIN = 60 

    resultados_finales = []
    historiales = []
    print("--- Iniciando Bucle Experimental ---")
    for lambda_val in lambdas_a_probar:
        print(f"\n--- Probando con λ = {lambda_val:.4f} ---")
        for i in range(N_REPETICIONES):
            resultado_run, df_hist = simular_una_corrida(lambda_val, warmup=WARMUP_MIN)
            resultado_run['lambda'] = lambda_val
            resultados_finales.append(resultado_run)
            df_hist['lambda'] = lambda_val
            df_hist['rep'] = i
            historiales.append(df_hist)
            print(".", end="")
    print("\n\n--- Bucle Experimental Finalizado ---")
    df_resultados = pd.DataFrame(resultados_finales)
    df_historiales = pd.concat(historiales, ignore_index=True)
    return df_resultados, df_historiales

if __name__ == '__main__':
    np.random.seed(42)
    random.seed(42)
    df_final_resultados_politica_4, df_hist = ejecutar_experimentos()

In [ ]:
df_estadisticas_politica_4 = df_final_resultados_politica_4.groupby('lambda').agg(
    n_simulaciones=('lambda', 'count'),
    total_aviones_promedio=('total_planes', 'mean'),
    cola_migraciones_promedio=('queue_size', 'mean'),
    cola_migraciones_std=('queue_size', 'std'),
    tiempo_espera_promedio=('wait_time_avg', 'mean'),
    tiempo_espera_std=('wait_time_avg', 'std'),
    tiempo_espera_domesticos_promedio=('wait_time_dom', 'mean'),
    tiempo_espera_internacionales_promedio=('wait_time_int', 'mean'),
    tiempo_espera_internacionales_std=('wait_time_int', 'std'),
    tiempo_espera_domesticos_std=('wait_time_dom', 'std'),
    cola_migraciones_domesticos_promedio=('queue_size_dom', 'mean'),
    cola_migraciones_domesticos_std=('queue_size_dom', 'std'),
    cola_migraciones_internacionales_promedio=('queue_size_int', 'mean'),
    cola_migraciones_internacionales_std=('queue_size_int', 'std')
)
# 2. Calcular el Error Estándar de la Media (nuestro error de estimación)
df_estadisticas_politica_4['tiempo_espera_error'] = 1.96 * df_estadisticas_politica_4['tiempo_espera_std'] / np.sqrt(df_estadisticas_politica_4['n_simulaciones'])
df_estadisticas_politica_4['tiempo_espera_domesticos_error'] = 1.96 * df_estadisticas_politica_4['tiempo_espera_domesticos_std'] / np.sqrt(df_estadisticas_politica_4['n_simulaciones'])
df_estadisticas_politica_4['tiempo_espera_internacionales_error'] = 1.96 * df_estadisticas_politica_4['tiempo_espera_internacionales_std'] / np.sqrt(df_estadisticas_politica_4['n_simulaciones'])
df_estadisticas_politica_4['cola_migraciones_domesticos_error'] = 1.96 * df_estadisticas_politica_4['cola_migraciones_domesticos_std'] / np.sqrt(df_estadisticas_politica_4['n_simulaciones'])
df_estadisticas_politica_4['cola_migraciones_internacionales_error'] = 1.96 * df_estadisticas_politica_4['cola_migraciones_internacionales_std'] / np.sqrt(df_estadisticas_politica_4['n_simulaciones'])
df_estadisticas_politica_4['cola_migraciones_error'] = 1.96 * df_estadisticas_politica_4['cola_migraciones_std'] / np.sqrt(df_estadisticas_politica_4['n_simulaciones'])

# Eliminar atraso y stds
df_estadisticas_politica_4 = df_estadisticas_politica_4.drop(columns=['cola_migraciones_std', 'tiempo_espera_std',
                                                 'tiempo_espera_domesticos_std', 'tiempo_espera_internacionales_std',
                                                 'cola_migraciones_domesticos_std', 'cola_migraciones_internacionales_std'])

# Mostramos la tabla final de resultados
print("--- Tabla de Estadísticas por Valor de λ ---")
display(df_estadisticas_politica_4)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True)
fig.suptitle('Impacto de implementación de política 4 en Migraciones', fontsize=18, weight='bold')

# --- Cola Domésticos ---
ax1 = axes[0,0]
ax1.plot(df_estadisticas.index, df_estadisticas['cola_migraciones_domesticos_promedio'], 
         marker='o', linestyle='--', color='royalblue', label='Día Normal')
ax1.plot(df_estadisticas_politica_4.index, df_estadisticas_politica_4['cola_migraciones_domesticos_promedio'], 
         marker='o', linestyle='-', color='deepskyblue', label='Con Política')
ax1.set_title('Cola Migraciones - Domésticos', fontsize=14)
ax1.set_ylabel('Tamaño Promedio de la Cola')
ax1.set_xlabel('Tasa de Arribo (λ)', fontsize=12)
ax1.legend()

# --- Cola Internacionales ---
ax2 = axes[0,1]
ax2.plot(df_estadisticas.index, df_estadisticas['cola_migraciones_internacionales_promedio'], 
         marker='s', linestyle='--', color='firebrick', label='Día Normal')
ax2.plot(df_estadisticas_politica_4.index, df_estadisticas_politica_4['cola_migraciones_internacionales_promedio'], 
         marker='s', linestyle='-', color='salmon', label='Con Política')
ax2.set_title('Cola Migraciones - Internacionales', fontsize=14)
ax2.set_ylabel('Tamaño Promedio de la Cola')
ax2.set_xlabel('Tasa de Arribo (λ)', fontsize=12)
ax2.legend()

# --- Espera Domésticos ---
ax3 = axes[1,0]
ax3.plot(df_estadisticas.index, df_estadisticas['tiempo_espera_domesticos_promedio'], 
         marker='o', linestyle='--', color='royalblue', label='Día Normal')
ax3.plot(df_estadisticas_politica_4.index, df_estadisticas_politica_4['tiempo_espera_domesticos_promedio'], 
         marker='o', linestyle='-', color='deepskyblue', label='Con Política')
ax3.set_title('Tiempo de Espera - Domésticos', fontsize=14)
ax3.set_ylabel('Minutos de Espera')
ax3.set_xlabel('Tasa de Arribo (λ)', fontsize=12)
ax3.legend()

# --- Espera Internacionales ---
ax4 = axes[1,1]
ax4.plot(df_estadisticas.index, df_estadisticas['tiempo_espera_internacionales_promedio'], 
         marker='s', linestyle='--', color='firebrick', label='Día Normal')
ax4.plot(df_estadisticas_politica_4.index, df_estadisticas_politica_4['tiempo_espera_internacionales_promedio'], 
         marker='s', linestyle='-', color='salmon', label='Con Política')
ax4.set_title('Tiempo de Espera - Internacionales', fontsize=14)
ax4.set_ylabel('Minutos de Espera')
ax4.set_xlabel('Tasa de Arribo (λ)', fontsize=12)
ax4.legend()

plt.tight_layout(rect=[0, 0, 1, 0.96])  # deja espacio para el título general
plt.show()
